## Напишіть функцію гіпотези лінійної регресії у векторному вигляді

$$h(x) = \omega_{0} + \omega_{1}x$$
$$h(x_{1},x_{2}) = \omega_{0} + \omega_{1}x_{1} + \omega_{2}x_{2}$$
$$h(x_{1},....,x_{n}) = \omega_{0} + \omega_{1}x_{1} + ... +\omega_{n}x_{n}$$
$$h(x_{i}) = \omega_{0} + \sum \limits _{i=1} ^{n} \omega_{n} x_{n}$$
$$ \vec{x} = \begin{pmatrix} 1 \\ x_{1} \\ \vdots \\ x_{2}\end{pmatrix} ,   \vec{\omega} = \begin{pmatrix} \omega_{0} \\ \omega_{1} \\ \vdots \\ \omega_{2}\end{pmatrix}$$
$$h(\vec{x}) = \vec{x} \cdot \vec{\omega}$$

In [8]:
import numpy as np

In [14]:
def linear_regression(x: np.array, w: np.array):
    assert x.shape[0] == w.shape[1], "Wrong shapes"
    return np.dot(x, w)

## Cтворіть функцію для обчислення функції втрат у векторному вигляді

$$ J(\omega_{0}, \omega_{}, ..., \omega_{n}) = \cfrac 1 {2m} \sum \limits _{i=1} ^{m} (h(x^{(i)} - y^{(i)}))^2 $$

In [ ]:
def loss(x: np.array, w: np.array, y: np.array):
    return np.sum(np.square(linear_regression(x, w) - y) / 2 * x.shape[0])

## Реалізуйте один крок градієнтного спуску


$$\Delta J (\vec \omega) = \cfrac 1 {m} X^{T} \Bigl (X \vec \omega -\vec y \Bigl)

In [ ]:
def gradient_step(x: np.array, w: np.array, y: np.array, learning_rate: float = 0.01):
    m= x.shape[0]
    delta = x.dot(w) - y
    step = np.dot(x.T, delta) / m
    return w - step*learning_rate

## Знайдіть найкращі параметри 𝑤⃗ для [датасету](https://drive.google.com/file/d/1-rAa4XT4_fI0dOBlMNuE6a7jB0wln_Qo/view?usp=share_link) прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень

In [2]:
import pandas as pd

In [4]:
URL = "https://drive.usercontent.google.com/download?id=1-rAa4XT4_fI0dOBlMNuE6a7jB0wln_Qo&export=download"

dataframe = pd.read_csv(URL, header=0)
if len(dataframe) == 0:
    print("Error: file not found")
    raise ValueError

dataframe


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


## Знайдіть ці ж параметри за допомогою аналітичного рішення

## Порівняйте отримані результати